In [101]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LogisticRegression
from tensorflow.examples.tutorials.mnist import input_data

In [102]:
#data
mnist = input_data.read_data_sets("MNIST-data", one_hot=False)
x_, y_ = mnist.train.next_batch(60000)
Y = []
X = []
for e in range(60000):
    if y_[e]<=1:
        Y.append(y_[e])
        X.append(x_[e])
X = np.asarray(X)
Y = np.asarray(Y)
print(X.shape)
        
x_test,y_test = mnist.test.next_batch(10000)
test_Y = []
test_X = []
for e in range(10000):
    if y_test[e]<=1:
        test_Y.append(y_test[e])
        test_X.append(x_test[e])
test_X = np.asarray(test_X)
test_Y = np.asarray(test_Y)
print(test_X.shape)

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
(12648, 784)
(2115, 784)


In [103]:
c1 = 0
c0 = 0
for l in Y:
    if l==0:
        c0 = c0+1
    elif l==1:
        c1 = c1+1
    
print(c1+c0)

12648


In [104]:
X = np.insert(X,len(X[0]),1,axis=1)
test_X = np.insert(test_X,len(test_X[0]),1,axis=1)
print(X.shape)

(12648, 785)


In [105]:
#constants
dim = 28
batch_size = 128
lr = 0.0005
iterations= 10000
train_total = len(X)
train_epochs = int(train_total/batch_size)
test_total = len(test_X)
test_epochs = int(test_total/batch_size)

#variables
W = np.ones((dim*dim+1,1))

In [106]:
#forward prop
def sigmoid(x):
    val = 1.+np.exp(-x)
    if val==0:
        print("error")
    return 1./val

def forward_prop(X,W):
    logits = np.dot(X,W)
    return sigmoid(logits)

#loss function and gradients
def BCE_loss(activations,labels):
    cost1 = labels*np.log(activations)
    cost2 = (1-labels)*np.log(1-activations)
    cost = cost1+cost2
    print(len(activations))
    cost = cost.sum()/len(activations)
    cost = -1*cost
    return cost

def gradient(p,t,X):
    error = p-t
    gradients = np.dot(error.T,X)
    print(len(p))
    gradients = gradients/len(p)
    gradients = gradients*lr
    return gradients

def update_rule(G,W):
    W = W - G.T
    return W

In [108]:
losses = []
itx = []
for i in range(iterations):
    loss = 0
    for j in range(train_epochs):
        train = X[batch_size*j:batch_size*(j+1)]
        labels = Y[batch_size*j:batch_size*(j+1)]
        predictions = forward_prop(train,W)
        g = gradient(predictions,labels,train)
        W = update_rule(g,W)
        loss = loss+BCE_loss(predictions,labels)
    losses.append(loss/batch_size)
    itx.append(i)
    if i%100==0:
        print(i, losses[-1])
        

12648
12648


C:\Users\ragha\Anaconda3\envs\tf1.12\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':
C:\Users\ragha\Anaconda3\envs\tf1.12\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in multiply
  if sys.path[0] == '':


12648
0 nan
12648
12648
12648


KeyboardInterrupt: 

In [ ]:
plt(itx,loss)

In [93]:
# Scikit Logistic Regression
clf = LogisticRegression()
clf.fit(X,Y)

#Score is Mean Accuracy
scikit_score = clf.score(test_X,test_Y)
print ('Scikit score: ', scikit_score)

C:\Users\ragha\Anaconda3\envs\tf1.12\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Scikit score:  0.9995271867612293
